# BERT with Global Classification & Sentence Contribution

# Imports & Installs

In [ ]:
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet
# !pip install tensorflow-text==2.15.0 --quiet
!pip install transformers==4.17 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.69 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 64.2 MB/s eta 0:00:00


In [ ]:
#@title Imports
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping
#import tensorflow_datasets as tfds
# import tensorflow_text as tf_text
import transformers

from transformers import logging
logging.set_verbosity_error()

import sklearn as sk
from sklearn.model_selection import train_test_split
import os

import nltk
from nltk.data import find
from nltk.tokenize import sent_tokenize
nltk.download('punkt_tab')

import matplotlib.pyplot as plt

import re

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from transformers import BertTokenizer, TFBertModel
# bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
# bert_model = TFBertModel.from_pretrained('bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Mounted at /content/drive


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

#### Read in data and tokenize

In [ ]:
# Read in data
final_data_folder = '/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/final_data/'
testing_data = pd.read_csv(final_data_folder + '/predictions/chunk_testing_predictions.csv')

# Keep only true positives
testing_data = testing_data[(testing_data['label'] == 1) & (testing_data['prediction'] == 1)]
testing_data

,Unnamed: 0,service_id,service_name,doc_type,doc_text,highlight,paraphrase,label,prediction
9,9,264,VKontakte,Privacy Policy,You should not share your credentials with thi...,You should not share your credentials with thi...,You are responsible for maintaining the securi...,1,1
13,13,616,Pornhub,Terms & Conditions,You are fully responsible for all activities t...,You are fully responsible for all activities t...,You are responsible for maintaining the securi...,1,1
15,15,403,Fur Affinity,Terms of Service,We retain the right to create limits on our se...,"We may change, suspend, or discontinue any or ...",They may stop providing the service at any time,1,1
17,17,2947,Skillshare,Terms of Service,We have the right to terminate your membership...,We have the right to terminate your membership...,Your account can be deleted without prior noti...,1,1
28,28,2544,El País,Privacy Policy,Se recomienda que el usuario facilite toda la ...,El Usuario tendr derecho a formular una queja ...,A complaint mechanism is provided for the hand...,1,1
...,...,...,...,...,...,...,...,...,...
5387,5387,3534,Flipboard,Privacy Policy,We use information as otherwise required or pe...,"We may use and disclose deidentified, anonymou...",This service shares your personal data with th...,1,1
5392,5392,460,credit_karma,Terms of Service,You are responsible for all activities that oc...,You are responsible for all activities that oc...,You are responsible for maintaining the securi...,1,1
5393,5393,1012,BetterHelp,Privacy Policy,A cookie is a small data file that is transfer...,We use cookies and web beacons to enable the t...,"This service requires first-party cookies, whi...",1,1
5396,5396,314,Quora,Privacy Policy,"For more information, see the section below ab...","For marketing and promotional purposes, such a...",This service may use your personal information...,1,1


In [ ]:
MAX_SEQUENCE_LENGTH = 512

def tokenize_with_separator(X, y):

  X_examples = [f" {bert_tokenizer.sep_token} ".join(sent_tokenize(x)) for x in X]

  bert_input_tokenized = bert_tokenizer(X_examples,
                                        max_length=MAX_SEQUENCE_LENGTH,
                                        truncation=True,
                                        padding='max_length',
                                        return_tensors='tf')

  bert_inputs = [bert_input_tokenized.input_ids,
                      bert_input_tokenized.token_type_ids,
                      bert_input_tokenized.attention_mask]

  bert_labels = np.array(y)

  return bert_inputs, bert_labels

In [ ]:
bert_test_inputs, bert_test_labels = tokenize_with_separator(testing_data['doc_text'], testing_data['label'])

#### Import the BERT classification model, and make predictions

In [ ]:
# Read in the model
stage_1_model_path = "/content/drive/MyDrive/266 Final Project/Full Orchestration: Stage 1+2/model_checkpoints/bert_chunk_classification_model/"
bert_chunk_model = tf.keras.models.load_model(stage_1_model_path)

# Attention Weight Comparison

In [ ]:
# Calculate the sentence contribution for each chunk -> mean sentence attention weights
def calculate_sentence_contribution_attention(bert_model, pooling, inputs, tokens, attention):

  bert_layer = bert_model.get_layer('bert')
  outputs = bert_layer([inputs, tokens, attention], output_attentions=True)
  attentions = outputs.attentions
  if pooling == 'mean':
    token_attention = tf.reduce_mean(attentions[-1][:, :, 0, :], axis=1)
  elif pooling == 'max':
    token_attention = tf.reduce_max(attentions[-1][:, :, 0, :], axis=1)
  else:
    token_attention = tf.reduce_sum(attentions[-1][:, :, 0, :], axis=1)

  # Identify the [SEP] token indeces for each input and store the indeces for all words between [SEP] tokens
  sep_tokens = np.where(inputs == bert_tokenizer.sep_token_id)[1]
  sentence_indices = [(1, sep_tokens[0])]
  for i in range(len(sep_tokens)-1):
    sentence_indices.append((sep_tokens[i] + 1, sep_tokens[i+1]))

  # Using the indeces found above, sum the attention weights for all tokens between the indeces
  sentence_contributions = []
  for start, end in sentence_indices:
    sentence_attention = tf.reduce_sum(token_attention[:, start:end], axis=1)
    sentence_contributions.append(sentence_attention.numpy()[0])

  # Normalize the aggregated sentence attention weights, to compare the importance of each sentence
  total_attention = sum(sentence_contributions)
  normalized_contributions = [x / total_attention for x in sentence_contributions]
  return normalized_contributions

In [ ]:
# Iterate through test examples, and calculate the sentence contribution using the fine-tuned bert model
sentence_contributions = []

for i in range(bert_test_inputs[0].shape[0]):
  contribution = calculate_sentence_contribution_attention(bert_chunk_model, 'mean', bert_test_inputs[0][i:i+1], bert_test_inputs[1][i:i+1], bert_test_inputs[2][i:i+1])
  sentence_contributions.append(contribution)

In [ ]:
highlight_ids = []
found = 0

for index, row in testing_data.iterrows():
  text = sent_tokenize(row['doc_text'])
  id = 0
  for i in text:
    if re.search(row['highlight'],i):
      found += 1
      break
    else:
      id += 1
  highlight_ids.append(id)

testing_data['highlight_id'] = highlight_ids

In [ ]:
testing_data['pred_sent_contribution_mean'] = sentence_contributions
testing_data['pred_highlight_id'] = testing_data['pred_sent_contribution_mean'].apply(lambda x: np.argmax(x))
testing_data['highlight_id_check'] = testing_data.apply(lambda x: x['highlight_id'] == x['pred_highlight_id'], axis = 1)

print('Attention weight comparison with mean pooling:')
testing_data['highlight_id_check'].value_counts()

Attention weight comparison with mean pooling:


,count
highlight_id_check,
False,643
True,345


In [ ]:
# Iterate through test examples, and calculate the sentence contribution using the fine-tuned bert model
sentence_contributions = []

for i in range(bert_test_inputs[0].shape[0]):
  contribution = calculate_sentence_contribution_attention(bert_chunk_model, 'max', bert_test_inputs[0][i:i+1], bert_test_inputs[1][i:i+1], bert_test_inputs[2][i:i+1])
  sentence_contributions.append(contribution)

testing_data['pred_sent_contribution_max'] = sentence_contributions
testing_data['pred_highlight_id'] = testing_data['pred_sent_contribution_max'].apply(lambda x: np.argmax(x))
testing_data['highlight_id_check'] = testing_data.apply(lambda x: x['highlight_id'] == x['pred_highlight_id'], axis = 1)

print('Attention weight comparison with max pooling:')
testing_data['highlight_id_check'].value_counts()

Attention weight comparison with max pooling:


,count
highlight_id_check,
False,655
True,333


In [ ]:
# Iterate through test examples, and calculate the sentence contribution using the fine-tuned bert model
sentence_contributions = []

for i in range(bert_test_inputs[0].shape[0]):
  contribution = calculate_sentence_contribution_attention(bert_chunk_model, 'sum', bert_test_inputs[0][i:i+1], bert_test_inputs[1][i:i+1], bert_test_inputs[2][i:i+1])
  sentence_contributions.append(contribution)

testing_data['pred_sent_contribution_sum'] = sentence_contributions
testing_data['pred_highlight_id'] = testing_data['pred_sent_contribution_sum'].apply(lambda x: np.argmax(x))
testing_data['highlight_id_check'] = testing_data.apply(lambda x: x['highlight_id'] == x['pred_highlight_id'], axis = 1)

print('Attention weight comparison with sum pooling:')
testing_data['highlight_id_check'].value_counts()

Attention weight comparison with sum pooling:


,count
highlight_id_check,
False,643
True,345


# Cosine Similarity with CLS

In [ ]:
# Calculate the sentence contribution for each chunk -> cosine similarity CLS token and sentence max sentence embeddings
def calculate_sentence_contribution_cosine(bert_model, inputs, tokens, attention):

  # Get embeddings for CLS token
  bert_layer = bert_model.get_layer('bert')
  outputs = bert_layer([inputs, tokens, attention])
  token_embeddings = outputs[0]
  cls = token_embeddings[:,0,:]
  cls = tf.nn.l2_normalize(cls,axis = -1)

  # Identify the [SEP] token indeces for each input and store the indeces for all words between [SEP] tokens
  sep_tokens = np.where(inputs == bert_tokenizer.sep_token_id)[1]
  sentence_indices = [(1, sep_tokens[0])]
  for i in range(len(sep_tokens)-1):
    sentence_indices.append((sep_tokens[i] + 1, sep_tokens[i+1]))

  # Using the indeces found above, iterate through sentences and get embeddings for each sentence using max pooling
  sentence_contributions = []
  for start, end in sentence_indices:
    sentence_embedding = tf.reduce_mean(token_embeddings[:, start:end], axis=1)
    sentence_embedding = tf.nn.l2_normalize(sentence_embedding, axis=-1)
    sentence_contributions.append(sentence_embedding.numpy()[0])

  # Calculate cosine similarity between CLS embeddings and sentence embeddings
  cosine_similarities = [(tf.reduce_sum(tf.multiply(cls, x))).numpy() for x in sentence_contributions]
  return cosine_similarities

In [ ]:
# Iterate through test examples, and calculate the sentence contribution using the fine-tuned bert model
sentence_contributions = []

for i in range(bert_test_inputs[0].shape[0]):
  contribution = calculate_sentence_contribution_cosine(bert_chunk_model, bert_test_inputs[0][i:i+1], bert_test_inputs[1][i:i+1], bert_test_inputs[2][i:i+1])
  sentence_contributions.append(contribution)

testing_data['pred_sent_contribution_cos'] = sentence_contributions
testing_data['pred_highlight_id'] = testing_data['pred_sent_contribution_cos'].apply(lambda x: np.argmax(x))
testing_data['highlight_id_check'] = testing_data.apply(lambda x: x['highlight_id'] == x['pred_highlight_id'], axis = 1)

print('Cosine Similarity betwen CLS and sentences:')
testing_data['highlight_id_check'].value_counts()

Cosine Similarity betwen CLS and sentences:


,count
highlight_id_check,
False,671
True,317


In [ ]:
testing_data.head(10)

,Unnamed: 0,service_id,service_name,doc_type,doc_text,highlight,paraphrase,label,prediction,highlight_id,pred_sentence_contributions,pred_highlight_id,highlight_id_check,pred_sent_contribution_mean,pred_sent_contribution_max,pred_sent_contribution_sum,pred_sent_contribution_cos
9,9,264,VKontakte,Privacy Policy,You should not share your credentials with thi...,You should not share your credentials with thi...,You are responsible for maintaining the securi...,1,1,0,"[0.18736143719384202, 0.3309179572402384, 0.48...",1,False,"[0.18736143719384202, 0.3309179572402384, 0.48...","[0.1375953896003676, 0.30015625270307905, 0.56...","[0.18736143963071014, 0.3309179615442373, 0.48...","[0.42832112, 0.43314743, 0.36607498]"
13,13,616,Pornhub,Terms & Conditions,You are fully responsible for all activities t...,You are fully responsible for all activities t...,You are responsible for maintaining the securi...,1,1,0,"[0.2733375478940585, 0.4497889914869472, 0.276...",0,True,"[0.2733375478940585, 0.4497889914869472, 0.276...","[0.17538693240247893, 0.6601071609108649, 0.16...","[0.2733375286665151, 0.4497890033883979, 0.276...","[0.3999401, 0.39059946, 0.37036228]"
15,15,403,Fur Affinity,Terms of Service,We retain the right to create limits on our se...,"We may change, suspend, or discontinue any or ...",They may stop providing the service at any time,1,1,2,"[0.4311800865696024, 0.423018536954323, 0.1458...",0,False,"[0.4311800865696024, 0.423018536954323, 0.1458...","[0.43252839307882535, 0.4371506248594908, 0.13...","[0.43118010893829173, 0.4230185130659413, 0.14...","[0.5801552, 0.5071672, 0.36502382]"
17,17,2947,Skillshare,Terms of Service,We have the right to terminate your membership...,We have the right to terminate your membership...,Your account can be deleted without prior noti...,1,1,0,"[0.6908429938271302, 0.06753771234751817, 0.24...",0,True,"[0.6908429938271302, 0.06753771234751817, 0.24...","[0.5169270196327838, 0.09076855128208199, 0.39...","[0.6908430139744977, 0.0675377116461823, 0.241...","[0.82277775, 0.7075685, 0.67919266]"
28,28,2544,El País,Privacy Policy,Se recomienda que el usuario facilite toda la ...,El Usuario tendr derecho a formular una queja ...,A complaint mechanism is provided for the hand...,1,1,1,"[0.11579140467771797, 0.03608713185817359, 0.8...",0,False,"[0.11579140467771797, 0.03608713185817359, 0.8...","[0.18951110276920186, 0.06411395505336913, 0.7...","[0.11579140487151016, 0.03608713497913185, 0.8...","[0.56707627, 0.5631834, 0.5293482]"
35,35,846,Crunchyroll,Terms of Service,You will cooperate fully with Ellation to inve...,The Site and Services may be used and accessed...,You agree not to use the service for illegal p...,1,1,2,"[0.3886049871740766, 0.25266404436403966, 0.35...",1,False,"[0.3886049871740766, 0.25266404436403966, 0.35...","[0.4282465817060757, 0.19568029867260117, 0.37...","[0.38860496917323684, 0.25266407388471684, 0.3...","[0.62523556, 0.64676386, 0.60314846]"
38,38,1638,Redbubble,Privacy Policy,"c your name, address, phone number, email addr...","c your name, address, phone number, email addr...",Details are provided about what kind of inform...,1,1,0,"[0.23569460935064002, 0.325598917489544, 0.438...",1,False,"[0.23569460935064002, 0.325598917489544, 0.438...","[0.49624258024287277, 0.1959993786878689, 0.30...","[0.2356946031709199, 0.325598945867276, 0.4387...","[0.43265808, 0.46120453, 0.45011112]"
43,43,2909,Replika,Terms of Service,Disclaimer of warranties\nYOUR USE OF THE SERV...,THE SERVICE IS PROVIDED ON AN AS IS AND AS AVA...,The service is provided 'as is' and to be used...,1,1,1,"[0.41761939975576845, 0.33416813843022586, 0.2...",0,False,"[0.41761939975576845, 0.33416813843022586, 0.2...","[0.42086324054644586, 0.3294298290883759, 0.24...","[0.4176193781051637, 0.3341681481621005, 0.248...","[0.47631708, 0.45574328, 0.34425458]"
48,48,214,WeTransfer,Terms of Service,The amended Terms will become effective upon t...,"Therefore, we recommend that you review the Te...","In

In [ ]:
testing_data['doc_text'][15]

'We retain the right to create limits on our services at any time with or without notice. We may also impose limits on services or aspects of them or restrict your access to part or all of our services without notice or liability. We may change, suspend, or discontinue any or all of our services at any time, including availability of any product, feature, database, and content yours or ours.'

In [ ]:
testing_data['highlight'][15]

'We may change, suspend, or discontinue any or all of our services at any time, including availability of any product, feature, database, and content yours or ours.'